## EIA V2 get_dataset examples

In [1]:
import gridstatus
import pandas as pd
import requests
import os
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

### Example for electricity/rto/region-data..

In [2]:
iso = gridstatus.EIA()
df = iso.get_dataset("electricity/rto/region-data", "Sep 15, 2023", "Sep 15, 2023")
print(df)

              Interval Start              Interval End Respondent  \
0  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       AECI   
1  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00        AVA   
2  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       AVRN   
3  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       AZPS   
4  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       BANC   
..                       ...                       ...        ...   
71 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       WACM   
72 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       WALC   
73 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       WAUW   
74 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00        WWA   
75 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00        YAD   

                                      Respondent Name  Load  Load Forecast  \
0               Associated Electric Cooperative, Inc.  2786           2799   
1              

### Let's graph this data

In [5]:
# Filter down to just the columns we want to graph
df_new = df[
    [
        "Interval Start",
        "Respondent",
        "Load",
        "Load Forecast",
        "Net Generation",
        "Total Interchange",
    ]
]
columns = ["Load", "Load Forecast", "Net Generation", "Total Interchange"]

# set up graph
fig = px.bar(
    df_new,
    x="Respondent",
    y=columns,
    title="EIA Generation Data for 9/15/2023",
)
# Update x-axis labels appearance
fig.update_xaxes(
    tickangle=45, tickfont=dict(size=10)
)  # Rotate and resize x-axis labels for better readability

# Adjust bar width
fig.update_layout(
    bargap=0.1,  # Adjust the gap between bars
    bargroupgap=0.05,  # Adjust the gap between groups of bars
)

# Show graph
fig.show(width=1200, height=900)

### Example for electricity/rto/interchange-data...

In [8]:
df = iso.get_dataset("electricity/rto/interchange-data", "Sep 15, 2023", "Sep 15, 2023")

# Lets check out the data..
print(df)

               Interval Start              Interval End From BA  \
0   2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00    AECI   
1   2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00    AECI   
2   2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00    AECI   
3   2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00    AECI   
4   2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00     AVA   
..                        ...                       ...     ...   
335 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00    WAUW   
336 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00    WAUW   
337 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00     WWA   
338 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00     YAD   
339 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00     YAD   

                                          From BA Name To BA  \
0                Associated Electric Cooperative, Inc.  MISO   
1                Associated Electric Cooperative, Inc.   SPA   
2  

### Graphing interchange data..

In [15]:
#  Filter down to just the columns we want
df_new = df[["Interval Start", "From BA", "To BA", "MW"]]

# Color the bars based on import/export (positive/negative values)
df["Color"] = np.where(df_new["MW"] < 0, "red", "green")

# Set up graph
fig = go.Figure()
fig.add_trace(
    go.Bar(
        name="EIA Interchange Data",
        x=df_new["From BA"],
        y=df_new["MW"],
        marker_color=df["Color"],
    )
)

# Update x-axis labels appearance
fig.update_xaxes(
    tickangle=45, tickfont=dict(size=8)
)  # Rotate and resize x-axis labels for better readability

# Adjust bar width
fig.update_layout(
    bargap=0.1,  # Adjust the gap between bars
    bargroupgap=0.05,  # Adjust the gap between groups of bars
)

# Add the "To BA" inside of the bars
fig.update_traces(texttemplate=df_new["To BA"], textposition="inside")

# Add graph titles
fig.update_layout(
    title="EIA Interchange Data for 9/15/2023",
    xaxis_title="From BA",
    yaxis_title="MW" "s",
)


# Show graph
# This graph shows the MWs imported/exported from the BA on the x-axis. Negative values represent an export
# the BA the power was sent/received from is included in the bars within the graph
fig.show(width=1800, height=900)

### Example for electricity/rto/fuel-type-data...

In [16]:
iso = gridstatus.EIA()
df = iso.get_dataset("electricity/rto/fuel-type-data", "Sep 15, 2023", "Sep 15, 2023")

# Lets check out the data..
print(df)

              Interval Start              Interval End Respondent  \
0  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       AECI   
1  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00        AVA   
2  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       AVRN   
3  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       AZPS   
4  2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       BANC   
..                       ...                       ...        ...   
71 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       WACM   
72 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       WALC   
73 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00       WAUW   
74 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00        WWA   
75 2023-09-14 23:00:00+00:00 2023-09-15 00:00:00+00:00        YAD   

                                      Respondent Name  Coal  Hydro  \
0               Associated Electric Cooperative, Inc.  1174      0   
1                              

### Now let's graph it..

In [19]:
# Filter down the data a bit..
df_new = df[
    [
        "Interval Start",
        "Respondent",
        "Coal",
        "Hydro",
        "Natural gas",
        "Nuclear",
        "Other",
        "Petroleum",
        "Solar",
        "Wind",
    ]
]
columns = [
    "Coal",
    "Hydro",
    "Natural gas",
    "Nuclear",
    "Other",
    "Petroleum",
    "Solar",
    "Wind",
]

# set up your graph
fig = px.bar(df_new, x="Respondent", y=columns, title="EIA Gen Mix Data for 9/15/23")

# Update x-axis labels appearance
fig.update_xaxes(
    tickangle=45, tickfont=dict(size=8)
)  # Rotate and resize x-axis labels for better readability

# Adjust bar width
fig.update_layout(
    bargap=0.1,  # Adjust the gap between bars
    bargroupgap=0.05,  # Adjust the gap between groups of bars
)

# Show graph
fig.show(width=1800, height=900)